In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [5]:
boston_data = pd.read_csv('Boston_crash.csv', skiprows=3)
cambridge_data = pd.read_csv('Cambridge_crash.csv', skiprows=3)
somerville_data = pd.read_csv('Somerville_Crash.csv', skiprows=3)

all_crash_data = pd.concat([boston_data, cambridge_data, somerville_data], ignore_index=True)
all_crash_data.head()

,Crash_Number,City_Town_Name,Crash_Date,Crash_Time,Crash_Severity,Maximum_Injury_Severity_Reported,Number_of_Vehicles,Total_Nonfatal_Injuries,Total_Fatal_Injuries,Manner_of_Collision,...,Ambient_Light,Weather_Condition,At_Roadway_Intersection,Distance_From_Nearest_Roadway_Intersection,Distance_From_Nearest_Milemarker,Distance_From_Nearest_Exit,Distance_From_Nearest_Landmark,Non_Motorist_Type,X_Cooordinate,Y_Cooordinate
0,4507366,BOSTON,01-Jan-2018,1:30 AM,Property damage only (none injured),No injury,2,0,0,"Sideswipe, opposite direction",...,Dark - lighted roadway,Clear/Clear,/ CHESTNUT HILL AVENUE,138 / CHESTNUT HILL AVENUE,NaN,NaN,NaN,NaN,228524.918696,899184.085223
1,4507934,BOSTON,01-Jan-2018,1:57 AM,Property damage only (none injured),No injury,1,0,0,Single vehicle crash,...,Dark - lighted roadway,Clear,L STREET / EMERSON STREET,L STREET / EMERSON STREET,NaN,NaN,NaN,VU2: Pedestrian,238292.422101,898312.875246
2,4516035,BOSTON,01-Jan-2018,2:50 AM,Non-fatal injury,Non-fatal injury - Possible,2,1,0,"Sideswipe, same direction",...,Dark - lighted roadway,Clear,RUTHERFORD AVENUE,RUTHERFORD AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
3,4482487,BOSTON,01-Jan-2018,2:53 AM,Property damage only (none injured),No injury,2,0,0,Angle,...,Dark - lighted roadway,Clear,VFW PARKWAY / WEST ROXBURY PARKWAY /,VFW PARKWAY / WEST ROXBURY PARKWAY /,NaN,NaN,NaN,NaN,229204.064716,894089.827050
4,4477254,BOSTON,01-Jan-2018,3:00 AM,Property damage only (none injured),No injury,4,0,0,Angle,...,Dark - lighted roadway,Cloudy,NaN,Rte 93 N,NaN,Exit 27 on Rte 93 N,NaN,NaN,235655.070210,902467.536218


In [10]:
import altair as alt
import pandas as pd
import warnings
warnings.filterwarnings("ignore", message=".*convert_dtype parameter is deprecated.*")

severity_mapping = {
    "No injury": "No Injury",
    "No Apparent Injury (O)": "No Injury",
    "Non-fatal injury - Possible": "Minor Injury",
    "Possible Injury (C)": "Minor Injury",
    "Non-fatal injury - Non-incapacitating": "Minor Injury",
    "Suspected Minor Injury (B)": "Minor Injury",
    "Non-fatal injury - Incapacitating": "Serious Injury",
    "Suspected Serious Injury (A)": "Serious Injury",
    "Fatal injury (K)": "Fatal",
    "Deceased not caused by crash": "Fatal",
    "Not reported": "Unknown",
    "Not Applicable": "Unknown",
    "Unknown": "Unknown"
}

# Apply mapping and remove "Unknown"
all_crash_data["Simplified_Severity"] = all_crash_data["Maximum_Injury_Severity_Reported"].map(severity_mapping)
cleaned_data = all_crash_data[~all_crash_data["Simplified_Severity"].isin(["Unknown"])]

# Define population for normalization
population = {
    "BOSTON": 675647,
    "CAMBRIDGE": 118403,
    "SOMERVILLE": 81045
}

# Group and normalize by population
grouped = cleaned_data.groupby(["City_Town_Name", "Simplified_Severity"]).size().reset_index(name="Crash_Count")
grouped["Population"] = grouped["City_Town_Name"].map(population)
grouped["Crashes_per_10k"] = (grouped["Crash_Count"] / grouped["Population"]) * 10000

# Define custom severity order
severity_order = ["No Injury", "Minor Injury", "Serious Injury", "Fatal"]

severity_dropdown = alt.binding_select(options=["All"] + severity_order, name="Select Severity: ")
selection = alt.param(name="severity_select", bind=severity_dropdown, value="All")

# Use blues color scheme with correct ordering
color_scale = alt.Scale(
    domain=severity_order,
    scheme="blues"
)

# Create the chart
chart = alt.Chart(grouped).mark_bar().encode(
    x=alt.X("City_Town_Name:N", title="City/Town Name"),
    y=alt.Y("Crashes_per_10k:Q", title="Crashes per 10,000 Residents"),
    color=alt.Color("Simplified_Severity:N", title="Injury Severity", scale=color_scale),
    tooltip=["City_Town_Name", "Simplified_Severity", "Crashes_per_10k"]
).add_params(
    selection
).transform_filter(
    (alt.datum.Simplified_Severity == selection) | (selection == "All")
).properties(
    width=800,
    height=600,
    title="Normalized Crash Injury Severity Across Boston Metropolitan Areas"
)

# Save the updated chart
chart.save("interactive_crash_severity_blues.html")